In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heartbeat/ptbdb_abnormal.csv
/kaggle/input/heartbeat/mitbih_train.csv
/kaggle/input/heartbeat/ptbdb_normal.csv
/kaggle/input/heartbeat/mitbih_test.csv


In [5]:
import wandb
from wandb.keras import WandbCallback

In [6]:
wandb.init()
config=wandb.config

wandb: Wandb version 0.9.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Network error (ConnectionError), entering retry loop. See /tmp/wandb-debug.log for full traceback.


In [7]:
#config.repeated_predictions=False
#config.look_back=20

In [8]:
config.epochs=3
config.batch_size=60
config.optimizer='rmsprop'

wandb: Wandb version 0.9.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Network error resolved after 0:00:58.067147, resuming normal operation.


In [9]:
data=pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv')
data.head()

,9.779411554336547852e-01,9.264705777168273926e-01,6.813725233078002930e-01,2.450980395078659058e-01,1.544117629528045654e-01,1.911764740943908691e-01,1.519607901573181152e-01,8.578431606292724609e-02,5.882352963089942932e-02,4.901960864663124084e-02,...,0.000000000000000000e+00.79,0.000000000000000000e+00.80,0.000000000000000000e+00.81,0.000000000000000000e+00.82,0.000000000000000000e+00.83,0.000000000000000000e+00.84,0.000000000000000000e+00.85,0.000000000000000000e+00.86,0.000000000000000000e+00.87,0.000000000000000000e+00.88
0,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.927461,1.000000,0.626943,0.193437,0.094991,0.072539,0.043178,0.053541,0.093264,0.189983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
data.shape

(87553, 188)

In [11]:
data_train=data[:int(data.shape[0]*0.8)]
data_train.shape

(70042, 188)

In [12]:
data_test=data[int(data.shape[0]*0.8):]
data_test.shape

(17511, 188)

In [13]:
train_x=data.iloc[:,:186]
train_x.shape

(87553, 186)

In [14]:
train_y=data.iloc[:,187]
train_y.shape

(87553,)

In [15]:
data_test=pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv')
data_test.head()

,1.000000000000000000e+00,7.582644820213317871e-01,1.115702465176582336e-01,0.000000000000000000e+00,8.057851344347000122e-02,7.851240038871765137e-02,6.611569970846176147e-02,4.958677664399147034e-02,4.752065986394882202e-02,3.512396663427352905e-02,...,0.000000000000000000e+00.56,0.000000000000000000e+00.57,0.000000000000000000e+00.58,0.000000000000000000e+00.59,0.000000000000000000e+00.60,0.000000000000000000e+00.61,0.000000000000000000e+00.62,0.000000000000000000e+00.63,0.000000000000000000e+00.64,0.000000000000000000e+00.65
0,0.908425,0.783883,0.531136,0.362637,0.366300,0.344322,0.333333,0.307692,0.296703,0.300366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.730088,0.212389,0.000000,0.119469,0.101770,0.101770,0.110619,0.123894,0.115044,0.132743,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.910417,0.681250,0.472917,0.229167,0.068750,0.000000,0.004167,0.014583,0.054167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.570470,0.399329,0.238255,0.147651,0.000000,0.003356,0.040268,0.080537,0.070470,0.090604,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.000000,0.923664,0.656489,0.195929,0.111959,0.175573,0.122137,0.050891,0.035623,0.055980,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
test_x=data_test.iloc[:,:186]
test_x.shape

(21891, 186)

In [17]:
test_y=data_test.iloc[:,187]
test_y.shape

(21891,)

In [18]:
import keras
train_y=keras.utils.to_categorical(train_y,num_classes=5)
test_y=keras.utils.to_categorical(test_y,num_classes=5)

Using TensorFlow backend.


In [19]:
train_x=np.array(train_x)
train_y=np.array(train_y)
test_x=np.array(test_x)
test_y=np.array(test_y)

Creating RNN Model

In [20]:
from keras.models import Sequential
from keras.layers import SimpleRNN,Embedding,Dense,LSTM,GRU

In [21]:
#model=Sequential()
#model.add(Embedding(10000,32))
#model.add(SimpleRNN(32,activation='sigmoid'))
#model.add(Dense(5,activation='softmax'))

#model.compile(optimizer=config.optimizer,loss='binary_crossentropy',metrics=['acc'])

In [22]:
#model.fit(train_x,train_y,epochs=config.epochs,batch_size=config.batch_size,callbacks=[WandbCallback()])

In [23]:
#using rnn model
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import accuracy_score
#pred=model.predict(test_x)
#accuracy=model.evaluate(test_x,test_y)
#print("accuracy is {}".format(accuracy[1]))
#print(confusion_matrix(pred,test_y))

Creating LSTM Model

In [24]:
model1=Sequential()
model1.add(Embedding(10000,32))
model1.add(LSTM(32,activation='sigmoid'))
model1.add(Dense(5,activation='softmax'))

model1.compile(optimizer=config.optimizer,loss='binary_crossentropy',metrics=['acc'])
model1.fit(train_x,train_y,epochs=config.epochs,batch_size=config.batch_size,callbacks=[WandbCallback()])

wandb: Wandb version 0.9.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/3
1460/1460 [==============================] - 122s 84ms/step - loss: 0.2338 - acc: 0.8167
Epoch 2/3
1460/1460 [==============================] - 128s 88ms/step - loss: 0.2257 - acc: 0.8277
Epoch 3/3
1460/1460 [==============================] - 125s 86ms/step - loss: 0.2256 - acc: 0.8277


In [25]:
#using lstm model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
pred=model1.predict(test_x)
accuracy=model1.evaluate(test_x,test_y)
print("accuracy is {}".format(accuracy[1]))
#print(confusion_matrix(pred,test_y))

685/685 [==============================] - 11s 16ms/step - loss: 0.2257 - acc: 0.8276
accuracy is 0.827600359916687
